In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import classification as clf
import imbalanced as imb
from sklearn.model_selection import KFold 
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, plot_roc_curve, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import statistics
data = clf.load_data("data_for_student_case.csv")

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image
from sklearn import tree
import matplotlib.pyplot as plt
import pylab as plt
import seaborn as sns



In [ ]:
pdata = clf.pre_process(data)
clf.describe_data(pdata)

pdata.head()

In [ ]:
pdata = imb.pre_process(data)
imb.describe_data(pdata)

# Get feature vector and truth vector from dataset
X, y = imb.get_X_y(pdata)

splits = 10
cv = KFold(n_splits=splits, shuffle=True)
scores_whitebox = np.zeros(10)
scores_blackbox = np.zeros(10)
false_negatives_white = np.zeros(10)
false_negatives_black = np.zeros(10)
true_positives_white = np.zeros(10)
true_positives_black = np.zeros(10)
feature_importance = np.zeros(11)
iteration = 0
targets = ['Non-fraud', 'Fraud']
for train_index, test_index in cv.split(X):
    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]

    # SMOTE Re-sampling
    sm = SMOTE()
    X_train, y_train = sm.fit_sample(X_train, y_train)

    # Classifiers
    # Decision Tree classifer (White-Box)
    clf = DecisionTreeClassifier()
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    importances = clf.feature_importances_
    for i in range(len(X_train.columns)):
        feature_importance[i] += importances[i]
    tree.plot_tree(clf.fit(X_test, y_test), max_depth = 5, feature_names = X_train.columns, class_names=targets, filled=True) 
    fig = plt.gcf()
    fig.set_size_inches(30, 20)
    fig.savefig('tree.png')

    CM = confusion_matrix(y_test, y_pred)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    true_positives_white[iteration] = TP
    false_negatives_white[iteration] = FN
    false_positve_rate = FP / (FP + TN)
    scores_whitebox[iteration] = false_positve_rate

    # Neural network model (Black-box)
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                        hidden_layer_sizes=(5, 2))
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    CM = confusion_matrix(y_test, y_pred)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    true_positives_black[iteration] = TP
    false_negatives_black[iteration] = FN
    false_positve_rate = FP / (FP + TN)
    scores_blackbox[iteration] = false_positve_rate
    iteration += 1 
print("Mean false positive rate (white): " + str(np.mean(scores_whitebox)))
print("Mean false positive rate (black): " + str(np.mean(scores_blackbox)))
print("Mean true positives (white): " + str(np.mean(true_positives_white)))

print("Mean true positives (black): " + str(np.mean(true_positives_black)))
print(true_positives_black)
print("Mean false positives (white): " + str(np.mean(false_negatives_white)))
print("Mean false positives (black): " + str(np.mean(false_negatives_black)))
print(false_negatives_black)

# Plot feature importance of the White-Box algorithms
feature_importance = np.divide(feature_importance, 10)
f, (ax) = plt.subplots(1,figsize =(22, 8))
ax = sns.barplot(x=X_train.columns,y=feature_importance)
ax.set(ylabel = "Normalized feature importance")
ax.set(xlabel = "Features")
plt.show()